In [12]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalAveragePooling2D
from keras.models import Model

# define the number of output classes
num_classes = 1

# create an instance of the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

# add a new fully connected layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='sigmoid')(x)

# create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# freeze the weights of the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# create an instance of the ImageDataGenerator class
datagen = ImageDataGenerator(rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
val_generator= datagen.flow_from_directory(
        'C:/newJupyterProject/kaveh_1402/MyProjects/tensorflow face recognition/people/testing',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

# load the training data for both datasets
train_generator = datagen.flow_from_directory(
        'C:/newJupyterProject/kaveh_1402/MyProjects/tensorflow face recognition/people/training',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

# load the training data for the second dataset
# val_generator2= datagen.flow_from_directory(
#         'C:/newJupyterProject/kaveh_1402/MyProjects/tensorflow face recognition/people/notkaveh0/notkavehtest',
#         target_size=(224, 224),
#         batch_size=32,
#         class_mode='binary')
# train_generator2 = datagen.flow_from_directory(
#         'C:/newJupyterProject/kaveh_1402/MyProjects/tensorflow face recognition/people/notkaveh0/notkavehtrain',
#         target_size=(224, 224),
#         batch_size=32,
#         class_mode='binary')

# train the model on both datasets
model.fit_generator(
        train_generator,
        steps_per_epoch=758//32,
        epochs=3,
        validation_data=val_generator,
        validation_steps=1)


Found 64 images belonging to 2 classes.
Found 758 images belonging to 2 classes.


c:\Users\kaveh\AppData\Local\anaconda3\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\kaveh\AppData\Local\Temp\ipykernel_9876\1381818809.py:65: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/3
23/23 [==============================] - 260s 11s/step - loss: 0.5989 - accuracy: 0.8926 - val_loss: 0.4889 - val_accuracy: 1.0000
Epoch 2/3
23/23 [==============================] - 240s 10s/step - loss: 0.4545 - accuracy: 0.9917 - val_loss: 0.3248 - val_accuracy: 1.0000
Epoch 3/3
23/23 [==============================] - 238s 10s/step - loss: 0.3408 - accuracy: 0.9848 - val_loss: 0.2513 - val_accuracy: 1.0000


In [15]:
from tensorflow.keras.utils import load_img,img_to_array
import numpy as np

# load the image
img_path = 'MyProjects/tensorflow face recognition/kavehselfie02.jpg'
img = load_img(img_path, target_size=(224, 224))

# convert the image to a numpy array
x = img_to_array(img)

# normalize the pixel values (rescale to between 0 and 1)
x = x / 255.0

# add a batch dimension to the array
x = np.expand_dims(x, axis=0)

# make the prediction
predictions = model.predict(x)
print(predictions)
# print the predicted label
if predictions[0] > 0.48:
    print("The image is a kaveh selfie.")
else:
    print("The image is not a kaveh selfie.")


1/1 [==============================] - 0s 348ms/step
[[0.72541386]]
The image is a kaveh selfie.


In [16]:
import tensorflow as tf
from keras.models import load_model

# load your Keras model
#model = load_model('my_model.h5')

# convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# save the TensorFlow Lite model to a file
with open('my_model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\kaveh\AppData\Local\Temp\tmp1q0eorcb\assets


INFO:tensorflow:Assets written to: C:\Users\kaveh\AppData\Local\Temp\tmp1q0eorcb\assets


In [17]:
# define the export path for TensorFlow Serving
export_path = 'C:/newJupyterProject/kaveh_1402/MyProjects'

# convert the Keras model to a TensorFlow SavedModel
tf.saved_model.save(model, export_path)

INFO:tensorflow:Assets written to: C:/newJupyterProject/kaveh_1402/MyProjects\assets


INFO:tensorflow:Assets written to: C:/newJupyterProject/kaveh_1402/MyProjects\assets


In [9]:
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="my_model.tflite")
interpreter.allocate_tensors()

# Get the input tensor index and shape
input_index = interpreter.get_input_details()[0]['index']
input_shape = interpreter.get_input_details()[0]['shape']

# Print the current input shape
print(f"Current input shape: {input_shape}")

# Update the input shape to (1, 224, 224, 3)
new_input_shape = (1, 224, 224, 3)
interpreter.resize_tensor_input(input_index, new_input_shape)
interpreter.allocate_tensors()
interpreter.invoke()

# Print the updated input shape
print(f"Updated input shape: {interpreter.get_input_details()[0]['shape']}")


Current input shape: [1 1 1 3]
Updated input shape: [  1 224 224   3]


In [10]:
interpreter = tf.lite.Interpreter(model_path="my_model.tflite")
interpreter.allocate_tensors()

# Get the input tensor index and shape
input_index = interpreter.get_input_details()[0]['index']
input_shape = interpreter.get_input_details()[0]['shape']

# Print the current input shape
print(f"Current input shape: {input_shape}")

Current input shape: [1 1 1 3]


In [2]:
# Save the modified model
with open('new_model.tflite', 'wb') as f:
    f.write(interpreter.model().serialize())


AttributeError: 'Interpreter' object has no attribute 'model'

In [6]:
import tensorflow as tf

# Load the TFLite model.
interpreter = tf.lite.Interpreter(model_path="my_model.tflite")
interpreter.allocate_tensors()

# Get input and output details.
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

# Modify the input shape.
interpreter.resize_tensor_input(input_details["index"], [1, 224, 224, 3])
interpreter.allocate_tensors()

# Save the modified model.
modified_model_path = "modified_model.tflite"
with open(modified_model_path, "wb") as f:
    f.write(interpreter._model_content.serialize())



AttributeError: 'Interpreter' object has no attribute '_model_content'